In [16]:
import pandas as pd
import numpy as np
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [17]:
#raw = importData('E29_CHECKIN1_MACROANALYSIS_V1.csv').replace("No Response", np.nan)

In [18]:
def importData(csv):
    data = pd.read_csv(csv)
    return data

#Break down goal setting and team process into: goals, support, communication, work allocation, team role, and enjoyment
#Then, find the magnitude of such difficulties for each team
def getCategoryDifferences(df, teamNumberCol):
    #Subset df to just goal setting and team process quantitative questions
    columnsOfInterest = [teamNumberCol, "SharedGoalTeamAvg", "SharedGoalClassAvg", 
                         "IndvGoalTeamAvg", "IndvGoalClassAvg", "SupportTeamAvg", 
                         "SupportClassAvg", "CommunicateTeamAvg", "CommunicateClassAvg", 
                         "WorkAllocTeamAvg", "WorkAllocClassAvg", "RoleTeamAvg", 
                         "RoleClassAvg", "EnjoyTeamAvg", "EnjoyClassAvg", 
                        'ConflictResTeamAvg', 'ConflictResClassAvg', 'ConflictResClassStDev',
                        'EfficiencyTeamAvg', 'EfficiencyClassAvg', 'EfficiencyClassStDev',
                        'ExpressionTeamAvg', 'ExpressionClassAvg', 'ExpressionClassStDev']
    temp = df[columnsOfInterest]
    
    #Collapse rows so that 1 row = 1 team
    temp = temp.drop_duplicates()
    
    #Calculate all averages (= team's cumulative deviation from class average)
    temp['goalsDiff'] = (temp["SharedGoalTeamAvg"] + temp["IndvGoalTeamAvg"]) - (temp["SharedGoalClassAvg"] + temp["IndvGoalClassAvg"])
    temp['supportDiff'] = temp["SupportTeamAvg"] - temp["SupportClassAvg"]
    temp['communicateDiff'] = temp["CommunicateTeamAvg"] - temp["CommunicateClassAvg"]
    temp['workAllocationDiff'] = temp["WorkAllocTeamAvg"] - temp["WorkAllocClassAvg"]
    temp['teamRoleDiff'] = temp["RoleTeamAvg"] - temp["RoleClassAvg"]
    temp['enjoyDiff'] = temp["EnjoyTeamAvg"] - temp["EnjoyClassAvg"]
    temp['conflictResDiff'] = temp["ConflictResTeamAvg"] - temp["ConflictResClassAvg"]
    temp['efficiencyDiff'] = temp['EfficiencyTeamAvg'] - temp['EfficiencyClassAvg']
    temp['expressionDiff'] = temp['ExpressionTeamAvg'] - temp['ExpressionClassAvg']
    temp['overallDiff'] = round(temp['goalsDiff'] + temp['supportDiff'] + temp['communicateDiff'] + temp['workAllocationDiff'] + temp['teamRoleDiff'] + temp['enjoyDiff'] + temp['conflictResDiff'] + temp['expressionDiff'] + temp['efficiencyDiff'], 2)
    
    tempRanked = temp.sort_values(teamNumberCol, ascending=True).reset_index(drop=True)
    tempRanked = tempRanked[[teamNumberCol, 'overallDiff', 'goalsDiff', 'supportDiff', 
                             'communicateDiff', 'workAllocationDiff', 'teamRoleDiff', 'enjoyDiff',
                            'conflictResDiff', 'efficiencyDiff', 'expressionDiff']]
    
    return tempRanked

#Assign each team a percentile (score) based on their overall difference from the class average. A difference of 0 designates a 50 (median)
def getTeamPercentiles(df, teamNumberCol):

    arr = np.array(df['overallDiff'])
    df["percentile"] = [round(stats.percentileofscore(arr, x), 2) for x in arr]

    return df
#Assign a category according to each team's percentile
def categorizePercentiles(df, teamNumberCol):
    arr = np.array(df['percentile'])
    cats = []
    #print(arr)
    for score in arr:
        if score >= 75:
            cats.append("Exceptional")
        elif score >= 50 and score < 75:
            cats.append("Strong")
        elif score >= 20 and score < 50:
            cats.append("Standard")
        elif score >= 10 and score < 20:
            cats.append("Weak")
        elif score >= 0 and score < 10:
            cats.append("Needs Attention")
        else:
            cats.append("Team did not respond")
    #print(cats)
    df["percentileCategory"] = cats
    return df

In [19]:
CHECKIN = pd.read_csv("HUTKA_CHECKIN01_MACROANALYSIS_V1.csv")

In [20]:
diffs = getCategoryDifferences(CHECKIN, 'TeamNumber')

In [21]:
percentiles = getTeamPercentiles(diffs, 'TeamNumber')

In [22]:
scored = categorizePercentiles(percentiles, 'TeamNumber')

In [23]:
scored.head()

,TeamNumber,overallDiff,goalsDiff,supportDiff,communicateDiff,workAllocationDiff,teamRoleDiff,enjoyDiff,conflictResDiff,efficiencyDiff,expressionDiff,percentile,percentileCategory
0,1,5.9,0.9,0.9,0.5,0.4,0.6,0.6,0.6,0.9,0.5,90.0,Exceptional
1,2,2.9,0.9,0.3,0.2,-0.4,0.9,0.3,0.1,0.4,0.2,60.0,Strong
2,3,4.4,1.5,0.6,0.2,0.4,0.3,0.3,0.3,0.6,0.2,70.0,Strong
3,5,7.6,1.5,0.9,0.8,1.2,0.6,0.6,0.6,0.9,0.5,100.0,Exceptional
4,7,0.8,0.5,0.8,-0.3,0.4,0.1,-0.2,0.1,-0.6,0.0,50.0,Strong


In [24]:
roster = pd.read_csv("HUTKA_ROSTER_V2.csv")
#roster = roster[["TeamName", "TeamNumber"]].drop_duplicates().reset_index()
macro = pd.read_csv("HUTKA_CHECKIN01_MACROANALYSIS_V1.csv")
scored = scored

In [26]:
scored.to_csv("HUTKA_CHECKIN01_TEAM_PERCENTILES.csv")

In [12]:
final = macro.merge(scored, on='TeamNumber').merge(roster, on='TeamNumber')

In [13]:
final

,Unnamed: 0,FullName,Email_x,TeamNumber,TeammateNumber_x,Our team is clear about the shared goals for our work together,SharedGoalTeamAvg,SharedGoalClassAvg,SharedGoalClassStDev,We each know about one another's individual goals for our work together,...,percentileCategory,FirstName,LastName,Email_y,TeammateNumber_y,Teammate1,Teammate2,Teammate3,Teammate4,Teammate5
0,0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,7.0,6.5,6.1,1.0,6.0,...,Exceptional,Mackenzie,Cramer,mackenzie.hanh@berkeley.edu,1,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
1,0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,7.0,6.5,6.1,1.0,6.0,...,Exceptional,Evan,Haas,evanhaas@berkeley.edu,2,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
2,0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,7.0,6.5,6.1,1.0,6.0,...,Exceptional,Dinah,Lee,jiwondinah.lee@berkeley.edu,3,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
3,0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,7.0,6.5,6.1,1.0,6.0,...,Exceptional,Catherine,McQueen,mcqueen@berkeley.edu,4,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
4,0,Mackenzie Cramer,mackenzie.hanh@berkeley.edu,1,1,7.0,6.5,6.1,1.0,6.0,...,Exceptional,Mindy,Tsai,mindytsai@berkeley.edu,5,Mackenzie Cramer,Evan Haas,Dinah Lee,Catherine McQueen,Mindy Tsai
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,42,Scarlett Xu,yiyao@berkeley.edu,18,5,7.0,6.2,6.1,1.0,7.0,...,Standard,Dan,Hickey,dan_hickey@berkeley.edu,1,Dan Hickey,Victoria Li,Mikah Nelson,Pamela Quartson,Scarlett Xu
183,42,Scarlett Xu,yiyao@berkeley.edu,18,5,7.0,6.2,6.1,1.0,7.0,...,Standard,Victoria,Li,vi2jia@berkeley.edu,2,Dan Hickey,Victoria Li,Mikah Nelson,Pamela Quartson,Scarlett Xu
184,42,Scarlett Xu,yiyao@berkeley.edu,18,5,7.0,6.2,6.1,1.0,7.0,...,Standard,Mikah,Nelson,mikah@berkeley.edu,3,Dan Hickey,Victoria Li,Mikah Nelson,Pamela Quartson,Scarlett Xu
185,42,Scarlett Xu,yiyao@berkeley.edu,18,5,7.0,6.2,6.1,1.0,7.0,...,Standard,Pamela,Quartson,pamela.quartson@berkeley.edu,4,Dan Hickey,Victoria Li,Mikah Nelson,Pamela Quartson,Scarlett Xu


In [14]:
final.to_csv("HUTKA_CHECKIN01_FULL_ANALYSIS_V1.csv")